<a href="https://colab.research.google.com/github/ggongch2/web-scarping/blob/main/%EC%9B%B9%ED%81%AC%EB%A1%A4%EB%A7%81%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 9.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#최초 1회
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks' #
!pip install chromedriver-autoinstaller


In [ ]:
https://velog.io/@kite_day/colab-%EC%97%90%EC%84%9C-%EC%9B%B9-%ED%81%AC%EB%A1%A4%EB%A7%81%ED%95%98%EA%B8%B0-selenium
https://m.blog.naver.com/sinx2233/223223626622 # selenium 응답 확인 seleniumwire
https://velog.io/@sangyeon217/selenium-webdriver-wait webdriverwait


In [2]:
from bs4 import BeautifulSoup, Tag
import requests
import re
import random
import sys
import selenium
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

In [3]:
chrome_path = "/content/drive/MyDrive/Colab Notebooks/chromedriver"
sys.path.insert(0,chrome_path)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
chrome_options.add_argument('lang=ko_KR') # 한국어


In [ ]:
# 랜덤문서에 방문해서 그문서에 기여한 사람 집계
# 다른 문서에 기여한 사람 집계 -> 모두 나라별
# 그래서 나라별 기여도가 비슷할수록 비슷한 문서/연관성이 높다고 볼수 있지않을까?

# 외부링크들을 모아서 Set에 넣고
#위키 재밌는 문서 찾기
# 랜덤문서 찾고 연관 문서가 다양한 것 ? 많은것?

In [53]:
def get_href_name(url) :
  if url is not None :
    if url.startswith("https://en.wikipedia.org/wiki/") :
      parts = url.split("/")
      wiki_index = parts.index("wiki")
      if "Wikipedia" not in parts[wiki_index+1] :
        return parts[wiki_index+1]
  else :
    return None
#cite도 없어야함, category 이것도 없어야함
#나머지 중복링크는 직접 수작업으로 처리

# 아래는 예시
urls = [
    "https://en.wikipedia.org/wiki/Geographic_coordinate_system",
    "https://en.wikipedia.org/wiki/New_South_Wales",
    "https://en.wikipedia.org/wiki/Main_Page",  # 정상적인 URL
    "https://en.wikipedia.org/",  # wiki가 없는 경우
    "https://www.example.com/page1", # 다른 URL
    "https://en.wikipedia.org/wiki/",
    "https://en.wikipedia.org/wiki/Wollemi_National_Park",
    "https://en.wikipedia.org/wiki/Wikipedia:Text_of_the_Creative_Commons_Attribution-ShareAlike_4.0_International_License",
    "https://foundation.wikimedia.org/wiki/Special:MyLanguage/Policy:Universal_Code_of_Conduct"

]


for url in urls :
  print(get_href_name(url))


Geographic_coordinate_system
New_South_Wales
Main_Page
None
None

Wollemi_National_Park
None
None


In [56]:
def get_href_set(driver) :
  href_set = set()
  a_tag = driver.find_elements(By.TAG_NAME, 'a')
  for each in a_tag :
    href_url = each.get_attribute('href')
    href_value = get_href_name(href_url)
    if href_value is not None and href_value not in basic_title :
      href_set.add(href_value)
  return href_set

# href 링크 모으기. class=new인건 수집x. /wiki/... 형식이여야함

In [20]:
def random_wiki_page() :
  url = "https://en.wikipedia.org/wiki/Special:Random"
  driver = webdriver.Chrome(options = chrome_options)
  driver.get(url)
  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  Title = driver.find_element(By.ID, 'firstHeading').text
  return driver

In [49]:
basic_title = list({'Special:RecentChangesLinked', 'Special:MyContributions', 'Main_Page', 'Special:RecentChanges', 'Special:Search', 'Special:Random', 'Special:SpecialPages', 'Help:Category', 'Help:Introduction', 'Help:Contents', 'Special:WhatLinksHere', 'Special:MyTalk', 'Portal:Current_events'})

In [55]:

basic_urls = list()

for each in basic_title :
  basic_urls.append("https://en.wikipedia.org/wiki/"+each)

print(basic_urls)

['https://en.wikipedia.org/wiki/Special:RecentChangesLinked', 'https://en.wikipedia.org/wiki/Special:MyContributions', 'https://en.wikipedia.org/wiki/Main_Page', 'https://en.wikipedia.org/wiki/Special:RecentChanges', 'https://en.wikipedia.org/wiki/Special:Search', 'https://en.wikipedia.org/wiki/Special:Random', 'https://en.wikipedia.org/wiki/Special:SpecialPages', 'https://en.wikipedia.org/wiki/Help:Category', 'https://en.wikipedia.org/wiki/Help:Introduction', 'https://en.wikipedia.org/wiki/Help:Contents', 'https://en.wikipedia.org/wiki/Special:WhatLinksHere', 'https://en.wikipedia.org/wiki/Special:MyTalk', 'https://en.wikipedia.org/wiki/Portal:Current_events']


In [57]:
driver = random_wiki_page()

print(get_href_set(driver))

{'The_Color_of_Me#cite_ref-ACC_4-2', 'Santa_Fe_Film_Festival', 'Category:2010s_English-language_films', 'The_Color_of_Me#cite_ref-11', 'The_Color_of_Me#cite_ref-8', 'The_Color_of_Me#cite_ref-ACC_4-3', 'The_Color_of_Me#Background', 'The_Color_of_Me#cite_ref-ACC_4-5', 'The_Color_of_Me#cite_note-24', 'Fantasy_film', 'The_Color_of_Me#cite_ref-DD_3-2', 'Indian_people', 'Category:Asian-American_drama_films', 'Nevada', 'The_Color_of_Me#cite_ref-10', 'The_Color_of_Me#cite_ref-12', 'Category:2018_drama_films', 'Category:2018_short_films', 'Amazon_Prime_Video', 'The_Color_of_Me#cite_ref-7', 'The_Color_of_Me#', 'Category:Template_film_date_with_1_release_date', 'The_Color_of_Me#cite_ref-1', 'The_Color_of_Me#cite_note-5', 'The_Color_of_Me', 'The_Color_of_Me#cite_ref-ACC_4-0', 'The_Color_of_Me#cite_note-18', 'The_Color_of_Me#cite_ref-14', 'Category:Hispanic_and_Latino_American_drama_films', 'The_Color_of_Me#Critical_reception_and_reviews', 'Category:English-language_fantasy_films', 'Category:Articl

In [58]:
def how_many_href(driver) :
  a_tag = driver.find_elements(By.TAG_NAME, 'a')
  return len(a_tag)